In [1]:
!pip show llama_index

Name: llama-index
Version: 0.12.8
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: /opt/anaconda3/envs/gradio/lib/python3.11/site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 


In [2]:
!pip install -q \
  llama-index \
  EbookLib \
  html2text \
  gradio \
  llama-index-embeddings-huggingface \
  llama-index-llms-ollama

In [22]:
import llama_index
import pkgutil

for importer, modname, ispkg in pkgutil.iter_modules(llama_index.__path__):
    print(modname)

_bundle
cli
core
legacy


In [3]:
import llama_index.core
print(dir(llama_index.core))

['BaseCallbackHandler', 'BasePromptTemplate', 'Callable', 'ChatPromptTemplate', 'ComposableGraph', 'Document', 'DocumentSummaryIndex', 'GPTDocumentSummaryIndex', 'GPTKeywordTableIndex', 'GPTListIndex', 'GPTRAKEKeywordTableIndex', 'GPTSimpleKeywordTableIndex', 'GPTTreeIndex', 'GPTVectorStoreIndex', 'IndexStructType', 'KeywordTableIndex', 'KnowledgeGraphIndex', 'ListIndex', 'MockEmbedding', 'NullHandler', 'Optional', 'Prompt', 'PromptHelper', 'PromptTemplate', 'PropertyGraphIndex', 'QueryBundle', 'RAKEKeywordTableIndex', 'Response', 'SQLContextBuilder', 'SQLDatabase', 'SQLDocumentContextBuilder', 'SelectorPromptTemplate', 'ServiceContext', 'Settings', 'SimpleDirectoryReader', 'SimpleKeywordTableIndex', 'StorageContext', 'SummaryIndex', 'TreeIndex', 'VectorStoreIndex', '__all__', '__annotations__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'async_utils', 'base', 'bridge', 'callbacks', 'chat_engine',

In [4]:
import llama_index.core
print(dir(llama_index.core))

['BaseCallbackHandler', 'BasePromptTemplate', 'Callable', 'ChatPromptTemplate', 'ComposableGraph', 'Document', 'DocumentSummaryIndex', 'GPTDocumentSummaryIndex', 'GPTKeywordTableIndex', 'GPTListIndex', 'GPTRAKEKeywordTableIndex', 'GPTSimpleKeywordTableIndex', 'GPTTreeIndex', 'GPTVectorStoreIndex', 'IndexStructType', 'KeywordTableIndex', 'KnowledgeGraphIndex', 'ListIndex', 'MockEmbedding', 'NullHandler', 'Optional', 'Prompt', 'PromptHelper', 'PromptTemplate', 'PropertyGraphIndex', 'QueryBundle', 'RAKEKeywordTableIndex', 'Response', 'SQLContextBuilder', 'SQLDatabase', 'SQLDocumentContextBuilder', 'SelectorPromptTemplate', 'ServiceContext', 'Settings', 'SimpleDirectoryReader', 'SimpleKeywordTableIndex', 'StorageContext', 'SummaryIndex', 'TreeIndex', 'VectorStoreIndex', '__all__', '__annotations__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'async_utils', 'base', 'bridge', 'callbacks', 'chat_engine',

In [5]:
import llama_index.core.node_parser
print(dir(llama_index.core.node_parser))

['CodeSplitter', 'HTMLNodeParser', 'HierarchicalNodeParser', 'JSONNodeParser', 'LangchainNodeParser', 'LanguageConfig', 'LlamaParseJsonNodeParser', 'MarkdownElementNodeParser', 'MarkdownNodeParser', 'MetadataAwareTextSplitter', 'NodeParser', 'SemanticDoubleMergingSplitterNodeParser', 'SemanticSplitterNodeParser', 'SentenceSplitter', 'SentenceWindowNodeParser', 'SimpleFileNodeParser', 'SimpleNodeParser', 'TextSplitter', 'TokenTextSplitter', 'UnstructuredElementNodeParser', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'file', 'get_child_nodes', 'get_deeper_nodes', 'get_leaf_nodes', 'get_root_nodes', 'interface', 'node_utils', 'relational', 'text']


In [6]:
cd "/Users/sylviathsu/Documents/COS243/LocalAILibrarian"

/Users/sylviathsu/Documents/COS243/LocalAILibrarian


In [7]:
import os
import logging
import gradio as gr
from llama_index.core import (
    SimpleDirectoryReader, 
    VectorStoreIndex, 
    StorageContext, 
    load_index_from_storage,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter
from pathlib import Path

/opt/anaconda3/envs/gradio/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def setup_logging():
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    return logger

logger = setup_logging()

# Step 2: Document Processing
def process_documents(doc_folder):
    reader = SimpleDirectoryReader(input_dir=doc_folder)
    documents = reader.load_data()
    splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
    nodes = splitter.get_nodes_from_documents(documents)
    return nodes

# Step 3: Embedding Generation
def generate_embeddings(nodes):
    embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
    vector_index = VectorStoreIndex.from_documents(nodes, embed_model=embed_model)
    storage_context = vector_index.storage_context
    storage_context.persist(persist_dir="storage")
    return vector_index

# Step 4: Query Engine Configuration
def configure_query_engine():
    llm = Ollama(model="phi3.5:3.8b-mini-instruct-q4_K_M")
    storage_context = StorageContext.from_defaults(persist_dir="storage")
    index = load_index_from_storage(storage_context, llm=llm)
    query_engine = index.as_query_engine(
        similarity_top_k=5,
        timeout=30  # Increase timeout to handle slow processing
    )
    return query_engine

# Step 5: Gradio Interface
def create_gradio_interface(query_engine):
    def query_docs(query, history):
        response = query_engine.query(query)
        sources = "\n".join([node.node.metadata.get('file_name', 'Unknown Source') for node in response.source_nodes])
        history.append((query, response.response + f"\nSources:\n{sources}"))
        return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history

    with gr.Blocks() as app:
        history = gr.State([])
        with gr.Row():
            gr.Markdown("## Local AI Librarian")
        with gr.Row():
            with gr.Column():
                query = gr.Textbox(label="Enter your query")
                submit_btn = gr.Button("Search")
            with gr.Column():
                output = gr.Textbox(label="Results", lines=20)
        submit_btn.click(query_docs, inputs=[query, history], outputs=[output, history])
    app.launch()

# Main Execution
if __name__ == "__main__":
    try:
        logger.info("Processing documents...")
        doc_folder = "./library"
        nodes = process_documents(doc_folder)

        logger.info("Generating embeddings and saving index...")
        vector_index = generate_embeddings(nodes)

        logger.info("Configuring query engine...")
        query_engine = configure_query_engine()

        logger.info("Launching Gradio interface...")
        create_gradio_interface(query_engine)

    except Exception as e:
        logger.error(f"An error occurred: {e}")

INFO:__main__:Processing documents...
INFO:__main__:Generating embeddings and saving index...
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
ERROR:__main__:An error occurred: 'TextNode' object has no attribute 'get_doc_id'


In [11]:
import os
import logging
import gradio as gr
from pathlib import Path
from typing import List, Tuple

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
    Settings
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter

def setup_logging():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

logger = setup_logging()

def process_documents(doc_folder: str):
    """Process documents from the specified folder"""
    try:
        reader = SimpleDirectoryReader(
            input_dir=doc_folder,
            recursive=True,
            filename_as_id=True,
            required_exts=[".txt", ".epub", ".pdf"]
        )
        return reader.load_data()
    except Exception as e:
        logger.error(f"Document processing error: {e}")
        raise

def generate_embeddings(documents):
    """Generate embeddings and create vector store index"""
    try:
        # Configure embedding model
        embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
        Settings.embed_model = embed_model
        
        # Configure text splitting
        splitter = SentenceSplitter(
            chunk_size=1024,
            chunk_overlap=200,
            paragraph_separator="\n\n",
            include_metadata=True
        )
        
        # Create and save index with transformations
        vector_index = VectorStoreIndex.from_documents(
            documents,
            transformations=[splitter],  # Apply transformations here
            embed_model=embed_model
        )
        
        storage_context = vector_index.storage_context
        storage_context.persist(persist_dir="storage")
        
        return vector_index
    except Exception as e:
        logger.error(f"Embedding generation error: {e}")
        raise

def configure_query_engine():
    """Configure the query engine with Ollama"""
    try:
        llm = Ollama(model="phi3.5:3.8b-mini-instruct-q4_K_M")
        Settings.llm = llm
        
        storage_context = StorageContext.from_defaults(persist_dir="storage")
        index = load_index_from_storage(storage_context)
        
        query_engine = index.as_query_engine(
            similarity_top_k=5,
            response_mode="tree_summarize",
            streaming=True,
            timeout=30
        )
        
        return query_engine
    except Exception as e:
        logger.error(f"Query engine configuration error: {e}")
        raise

def create_gradio_interface(query_engine):
    """Create and configure the Gradio interface"""
    def query_docs(query: str, history: List[Tuple[str, str]]) -> Tuple[str, List]:
        try:
            response = query_engine.query(query)
            
            # Extract source information
            sources = []
            if hasattr(response, 'source_nodes'):
                sources = [
                    f"- {node.node.metadata.get('file_name', 'Unknown')}"
                    for node in response.source_nodes
                ]
            
            source_text = "\nSources:\n" + "\n".join(sources) if sources else ""
            full_response = str(response) + source_text
            
            history.append((query, full_response))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history
            
        except Exception as e:
            logger.error(f"Query processing error: {e}")
            error_msg = f"Error processing query: {str(e)}"
            history.append((query, error_msg))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history

    with gr.Blocks(title="Local AI Librarian") as app:
        gr.Markdown("# Local AI Librarian")
        gr.Markdown("Search your document collection using natural language queries.")
        
        with gr.Row():
            with gr.Column(scale=2):
                query_input = gr.Textbox(
                    label="Enter your query",
                    placeholder="What would you like to know about your documents?"
                )
                examples = gr.Examples(
                    examples=[
                        "What are the main themes in the documents?",
                        "Summarize the key points about...",
                        "Find relevant passages about..."
                    ],
                    inputs=query_input
                )
                
        with gr.Row():
            submit_btn = gr.Button("Search")
            clear_btn = gr.Button("Clear History")
            
        chat_history = gr.State([])
        output = gr.Textbox(
            label="Results",
            lines=20,
            autoscroll=False
        )
        
        submit_btn.click(
            query_docs,
            inputs=[query_input, chat_history],
            outputs=[output, chat_history]
        )
        clear_btn.click(
            lambda: ([], []),
            outputs=[output, chat_history]
        )
        
    return app

if __name__ == "__main__":
    try:
        # Step 1: Set up logging
        logger.info("Starting application...")
        
        # Step 2: Check if index exists
        if not Path("storage").exists():
            logger.info("Processing documents...")
            doc_folder = "./library"
            documents = process_documents(doc_folder)
            
            logger.info("Generating embeddings and saving index...")
            vector_index = generate_embeddings(documents)
        
        # Step 3: Configure query engine
        logger.info("Configuring query engine...")
        query_engine = configure_query_engine()
        
        # Step 4: Launch Gradio interface
        logger.info("Launching Gradio interface...")
        app = create_gradio_interface(query_engine)
        app.launch(share=False)
        
    except Exception as e:
        logger.error(f"Application error: {e}")
        raise

INFO:__main__:Starting application...
INFO:__main__:Processing documents...
INFO:__main__:Generating embeddings and saving index...
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
Batches: 100%|██████████| 1/1 [00:01<00:00,  1.92s/it]
INFO:__main__:Configuring query engine...
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:__main__:Launching Gradio interface...
INFO:httpx:HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "
INFO:httpx:HTTP Request: GET http://127.0.0.1:7860/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
Batches: 100%|██████████| 1/1 [00:01<00:00,  1.89s/it]
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


In [15]:
import os
import logging
import gradio as gr
from pathlib import Path
import shutil
from typing import List, Tuple

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter

def setup_logging():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

logger = setup_logging()

def process_documents(doc_folder: str):
    """Process documents from the specified folder"""
    try:
        reader = SimpleDirectoryReader(
            input_dir=doc_folder,
            recursive=True,
            filename_as_id=True,
            required_exts=[".txt", ".epub", ".pdf"]
        )
        return reader.load_data()
    except Exception as e:
        logger.error(f"Document processing error: {e}")
        raise

def generate_embeddings(documents):
    """Generate embeddings and create vector store index"""
    try:
        # Configure embedding model
        embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
        
        # Configure text splitting
        splitter = SentenceSplitter(
            chunk_size=512,
            chunk_overlap=50,
            paragraph_separator="\n\n",
            include_metadata=True
        )
        
        # Create and save index
        vector_index = VectorStoreIndex.from_documents(
            documents,
            embed_model=embed_model,
            transformations=[splitter],
            show_progress=True
        )
        
        vector_index.storage_context.persist(persist_dir="storage")
        return vector_index
    except Exception as e:
        logger.error(f"Embedding generation error: {e}")
        raise

def configure_query_engine():
    """Configure the query engine with optimized parameters"""
    try:
        # Configure LLM with increased timeout
        llm = Ollama(
            model="phi3.5:3.8b-mini-instruct-q4_K_M",
            request_timeout=120.0,
            temperature=0.1
        )
        
        storage_context = StorageContext.from_defaults(persist_dir="storage")
        index = load_index_from_storage(storage_context)
        
        query_engine = index.as_query_engine(
            llm=llm,
            similarity_top_k=3,
            response_mode="compact",
            streaming=False
        )
        
        return query_engine
    except Exception as e:
        logger.error(f"Query engine configuration error: {e}")
        raise

def create_gradio_interface():
    """Create and configure the Gradio interface"""
    # Initialize query engine at the start
    global_query_engine = configure_query_engine() if Path("storage").exists() else None
    
    def handle_file_upload(files, progress=gr.Progress()):
        """Handle file upload and index update"""
        try:
            nonlocal global_query_engine
            
            # Create library directory if it doesn't exist
            Path("library").mkdir(exist_ok=True)
            
            # Copy uploaded files to library directory
            progress(0, desc="Copying files...")
            for file in files:
                shutil.copy2(file.name, "library")
            
            # Process documents and update index
            progress(0.3, desc="Processing documents...")
            documents = process_documents("library")
            
            progress(0.6, desc="Generating embeddings...")
            generate_embeddings(documents)
            
            progress(0.9, desc="Configuring query engine...")
            global_query_engine = configure_query_engine()
            
            progress(1.0, desc="Done!")
            return "Files uploaded and indexed successfully!"
        except Exception as e:
            logger.error(f"Upload error: {e}")
            return f"Error processing files: {str(e)}"

    def query_docs(query: str, history: List[Tuple[str, str]]) -> Tuple[str, List]:
        nonlocal global_query_engine
        
        if global_query_engine is None:
            error_msg = "Please upload documents first or wait for indexing to complete."
            history.append((query, error_msg))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history
        
        try:
            response = global_query_engine.query(query)
            
            # Extract source information
            sources = []
            if hasattr(response, 'source_nodes'):
                sources = [
                    f"- {node.node.metadata.get('file_name', 'Unknown')}"
                    for node in response.source_nodes
                ]
            
            source_text = "\nSources:\n" + "\n".join(sources) if sources else ""
            full_response = str(response) + source_text
            
            history.append((query, full_response))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history
            
        except Exception as e:
            logger.error(f"Query processing error: {e}")
            error_msg = f"Error processing query: {str(e)}"
            history.append((query, error_msg))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history

    with gr.Blocks(title="Local AI Librarian") as app:
        gr.Markdown("# Local AI Librarian")
        gr.Markdown("Upload documents and search through them using natural language queries.")
        
        # Document upload section
        with gr.Row():
            with gr.Column():
                file_output = gr.Textbox(label="Upload Status")
                upload_button = gr.File(
                    file_count="multiple",
                    label="Upload Documents",
                    file_types=[".txt", ".pdf", ".epub"]
                )
        
        # Query section
        with gr.Row():
            with gr.Column(scale=2):
                query_input = gr.Textbox(
                    label="Enter your query",
                    placeholder="What would you like to know about your documents?"
                )
                examples = gr.Examples(
                    examples=[
                        "What are the main themes in the documents?",
                        "Summarize the key points about...",
                        "Find relevant passages about..."
                    ],
                    inputs=query_input
                )
                
        with gr.Row():
            submit_btn = gr.Button("Search")
            clear_btn = gr.Button("Clear History")
            
        chat_history = gr.State([])
        output = gr.Textbox(
            label="Results",
            lines=20,
            autoscroll=False
        )
        
        # Set up event handlers
        upload_button.upload(
            handle_file_upload,
            inputs=[upload_button],
            outputs=[file_output]
        )
        
        submit_btn.click(
            query_docs,
            inputs=[query_input, chat_history],
            outputs=[output, chat_history]
        )
        
        clear_btn.click(
            lambda: ([], []),
            outputs=[output, chat_history]
        )
        
    return app

if __name__ == "__main__":
    try:
        logger.info("Starting application...")
        
        # Launch Gradio interface
        logger.info("Launching Gradio interface...")
        app = create_gradio_interface()
        app.launch(share=False)
        
    except Exception as e:
        logger.error(f"Application error: {e}")
        raise

INFO:__main__:Starting application...
INFO:__main__:Launching Gradio interface...
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:httpx:HTTP Request: GET http://127.0.0.1:7863/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7863/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
Generating embeddings: 100%|██████████| 662/662 [01:30<00:00,  7.30it/s]
INFO:llama_index.core.indices.loading:Loading all indices.
Batches: 100%|██████████| 1/1 [00:01<00:00,  1.09s/it]
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


In [16]:
import os
import logging
import gradio as gr
from pathlib import Path
import shutil
from typing import List, Tuple
import json
from datetime import datetime

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter

def setup_logging():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

logger = setup_logging()

# [Previous functions remain unchanged: process_documents, generate_embeddings, configure_query_engine]

def save_conversation_history(history: List[Tuple[str, str]], file_format: str = "txt") -> str:
    """Save conversation history to a file"""
    try:
        # Create exports directory if it doesn't exist
        export_dir = Path("exports")
        export_dir.mkdir(exist_ok=True)
        
        # Generate timestamp for filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        if file_format == "json":
            # Save as JSON
            filename = export_dir / f"conversation_{timestamp}.json"
            conversation_data = {
                "timestamp": timestamp,
                "messages": [{"question": q, "answer": a} for q, a in history]
            }
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(conversation_data, f, indent=2, ensure_ascii=False)
        else:
            # Save as plain text
            filename = export_dir / f"conversation_{timestamp}.txt"
            with open(filename, 'w', encoding='utf-8') as f:
                for q, a in history:
                    f.write(f"Q: {q}\n")
                    f.write(f"A: {a}\n")
                    f.write("-" * 80 + "\n")
        
        return f"Conversation saved to {filename}"
    except Exception as e:
        logger.error(f"Error saving conversation: {e}")
        return f"Error saving conversation: {str(e)}"

def create_gradio_interface():
    """Create and configure the Gradio interface"""
    # Initialize query engine at the start
    global_query_engine = configure_query_engine() if Path("storage").exists() else None
    
    def handle_file_upload(files, progress=gr.Progress()):
        """Handle file upload and index update"""
        try:
            nonlocal global_query_engine
            
            # Create library directory if it doesn't exist
            Path("library").mkdir(exist_ok=True)
            
            # Copy uploaded files to library directory
            progress(0, desc="Copying files...")
            for file in files:
                shutil.copy2(file.name, "library")
            
            # Process documents and update index
            progress(0.3, desc="Processing documents...")
            documents = process_documents("library")
            
            progress(0.6, desc="Generating embeddings...")
            generate_embeddings(documents)
            
            progress(0.9, desc="Configuring query engine...")
            global_query_engine = configure_query_engine()
            
            progress(1.0, desc="Done!")
            return "Files uploaded and indexed successfully!"
        except Exception as e:
            logger.error(f"Upload error: {e}")
            return f"Error processing files: {str(e)}"

    def query_docs(query: str, history: List[Tuple[str, str]]) -> Tuple[str, List]:
        nonlocal global_query_engine
        
        if global_query_engine is None:
            error_msg = "Please upload documents first or wait for indexing to complete."
            history.append((query, error_msg))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history
        
        try:
            response = global_query_engine.query(query)
            
            # Extract source information
            sources = []
            if hasattr(response, 'source_nodes'):
                sources = [
                    f"- {node.node.metadata.get('file_name', 'Unknown')}"
                    for node in response.source_nodes
                ]
            
            source_text = "\nSources:\n" + "\n".join(sources) if sources else ""
            full_response = str(response) + source_text
            
            history.append((query, full_response))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history
            
        except Exception as e:
            logger.error(f"Query processing error: {e}")
            error_msg = f"Error processing query: {str(e)}"
            history.append((query, error_msg))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history

    with gr.Blocks(title="Local AI Librarian") as app:
        gr.Markdown("# Local AI Librarian")
        gr.Markdown("Upload documents and search through them using natural language queries.")
        
        # Document upload section
        with gr.Row():
            with gr.Column():
                file_output = gr.Textbox(label="Upload Status")
                upload_button = gr.File(
                    file_count="multiple",
                    label="Upload Documents",
                    file_types=[".txt", ".pdf", ".epub"]
                )
        
        # Query section
        with gr.Row():
            with gr.Column(scale=2):
                query_input = gr.Textbox(
                    label="Enter your query",
                    placeholder="What would you like to know about your documents?"
                )
                examples = gr.Examples(
                    examples=[
                        "What are the main themes in the documents?",
                        "Summarize the key points about...",
                        "Find relevant passages about..."
                    ],
                    inputs=query_input
                )
                
        with gr.Row():
            submit_btn = gr.Button("Search")
            clear_btn = gr.Button("Clear History")
            
        # Export controls
        with gr.Row():
            export_format = gr.Radio(
                choices=["txt", "json"],
                value="txt",
                label="Export Format"
            )
            export_btn = gr.Button("Export Conversation")
            export_status = gr.Textbox(label="Export Status")
            
        chat_history = gr.State([])
        output = gr.Textbox(
            label="Results",
            lines=20,
            autoscroll=False
        )
        
        # Set up event handlers
        upload_button.upload(
            handle_file_upload,
            inputs=[upload_button],
            outputs=[file_output]
        )
        
        submit_btn.click(
            query_docs,
            inputs=[query_input, chat_history],
            outputs=[output, chat_history]
        )
        
        clear_btn.click(
            lambda: ([], []),
            outputs=[output, chat_history]
        )
        
        export_btn.click(
            save_conversation_history,
            inputs=[chat_history, export_format],
            outputs=[export_status]
        )
        
    return app

if __name__ == "__main__":
    try:
        logger.info("Starting application...")
        
        # Launch Gradio interface
        logger.info("Launching Gradio interface...")
        app = create_gradio_interface()
        app.launch(share=False)
        
    except Exception as e:
        logger.error(f"Application error: {e}")
        raise

INFO:__main__:Starting application...
INFO:__main__:Launching Gradio interface...
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:httpx:HTTP Request: GET http://127.0.0.1:7864/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7864/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
Generating embeddings: 100%|██████████| 662/662 [01:19<00:00,  8.36it/s]
INFO:llama_index.core.indices.loading:Loading all indices.
Batches: 100%|██████████| 1/1 [00:02<00:00,  2.68s/it]
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
